## Imports

In [93]:
import numpy as np
import gymnasium as gym
from IPython.display import HTML
from base64 import b64encode
import imageio

## Utils

In [94]:
def record_video(env, policy, out_directory, fps=1, random_action=False, max_steps=100):
    images = []  
    done = False
    truncated = False
    state, info = env.reset()
    img = env.render()
    images.append(img)
    total_reward = 0
    i = 0
    while not done and not truncated:
        i += 1
        if i > max_steps:
            break
        action = np.random.randint(4) if random_action else policy[state]
        state, reward, done, truncated, info = env.step(action)
        total_reward += reward
        img = env.render()
        images.append(img)
        if not random_action:
            print(f"action: {action}, state: {state}, reward: {reward}, done: {done}, truncated: {truncated}, info: {info}")
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)
    return total_reward

In [95]:
def show_video(video_path, video_width=500):
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

## Random Walk

In [96]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode='rgb_array')
total_reward = record_video(env, None, 'frozenlake_random.mp4', fps=3, random_action=True)
print(f"total reward: {total_reward}")
show_video('frozenlake_random.mp4', video_width=500)

total reward: 0.0


## Define FrozenLake MDP

In [97]:
class FrozenLakeMDP:
    def __init__(self, is_slippery):
        self.is_slippery = is_slippery
        self.terminal_states = np.zeros(16, dtype=int)
        self.terminal_states[[5, 7, 11, 12, 15]] = 1
        self.reward_fn = np.zeros(16, dtype=int)
        self.reward_fn[15] = 1

    def is_terminal(self, state):
        return self.terminal_states[state]
    
    def get_reward_function(self):
        return self.reward_fn

    def next_state_det(self, state, action):
        if action == 0:    # LEFT
            next_state = state - 1 if state % 4 != 0 else state
        elif action == 1:  # DOWN
            next_state = state + 4 if state // 4 != 3 else state
        elif action == 2:  # RIGHT
            next_state = state + 1 if state % 4 != 3 else state
        elif action == 3:  # UP
            next_state = state - 4 if state // 4 != 0 else state
        else:         # WRONG ACTION
            next_state = state
        return next_state
    
    def trans_prob(self, state, action):
        prob = np.zeros((16,), dtype=float)
        if not self.is_slippery:
            prob[self.next_state_det(state, action)] = 1.0
        else:
            prob[self.next_state_det(state, action)] += 1/3
            prob[self.next_state_det(state, (action+1)%4)] += 1/3
            prob[self.next_state_det(state, (action-1)%4)] += 1/3
        return prob

    def next_state_reward(self, state, action):
        next_state_probs = self.trans_prob(state, action)
        next_state = np.random.choice(16, p=next_state_probs)
        reward = self.reward_fn[next_state]
        return next_state, reward

In [98]:
dynamics = FrozenLakeMDP(is_slippery=False)

In [99]:
# reward function of the environent
dynamics.get_reward_function()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [100]:
# evaluating if a given state is a terminal state (= hole or goal)
print(dynamics.is_terminal(0), dynamics.is_terminal(7), dynamics.is_terminal(15))

0 1 1


In [101]:
# if we take action `a` in state `s`,
# what is the probability of landing in each state?
dynamics.trans_prob(10, 1)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [102]:
# if we take action `a` in state `s`, what do we get?
# this is done through sampling the transition probability
next_state, reward = dynamics.next_state_reward(10, 1)
print(next_state, reward)

14 0


## Iterative Policy Evaluation

In [146]:
def policy_evaluation(dynamics, policy,  num_iter, gamma=0.9):
    """
    evaluates policy based on Iterative Policy Evaluation.
 
    Args:
        dynamics (FrozenLakeMDP): dynamics of the environment.
        policy (numpy.array): policy we want to evaluate.
        gamma (float): discount factor.
        num_iter (int): number of iterations for the loop.
 
    Returns:
        numpy.array: state value function.
    """

    # TODO: implement Iterative Policy Evaluation algorithm

    s_value_function = np.zeros(16, dtype=float)
    rewards = dynamics.get_reward_function()
    for k in range(num_iter):
        for s in range(0, 16):
            if dynamics.is_terminal(s):
                continue
            prob = dynamics.trans_prob(s, policy[s])
            expected = 0
            for ns in range(0, 16):
                expected += (prob[ns] * (rewards[ns] + gamma * s_value_function[ns]))
            s_value_function[s] = expected
    return s_value_function

In [147]:
dynamics = FrozenLakeMDP(is_slippery=False)

# 1. go-right policy
policy1 = 2 * np.ones(16, dtype=int)

# 2. shortest-path policy
policy2 = np.array([1, 2, 1, 0, 1, -1, 1, -1, 2, 1, 1, -1, -1, 2, 2, -1])

s_value_function1 = policy_evaluation(dynamics, policy1, 10)
s_value_function2 = policy_evaluation(dynamics, policy2, 10)

In [148]:
# TODO: print and analyze the state value function
print(s_value_function1)
print(s_value_function2)

[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.9 1.  0. ]
[0.59049 0.6561  0.729   0.6561  0.6561  0.      0.81    0.      0.729
 0.81    0.9     0.      0.      0.9     1.      0.     ]


## Policy Iteration

In [149]:
def greedy_policy_improvement(dynamics, s_value_function, gamma=0.9):
    """
    obtains a policy in a greedy manner based on current state value function.4
 
    Args:
        dynamics (FrozenLakeMDP): dynamics of the environment.
        s_value_function (numpy.array): state value function.
        gamma (float): discount factor.
 
    Returns:
        numpy.array: the greedy policy.
    """

    # TODO: implement Greedy Policy Improvement algorithm
    policy = np.random.randint(0, 4, size=16) 
    rewards = dynamics.get_reward_function()
    for s in range(0, 16):
        if dynamics.is_terminal(s):
                continue
        temp_array = np.zeros(4, dtype=float)
        for a in range(0, 4):
            prob = dynamics.trans_prob(s, a)
            for ns in range(0, 16):
                temp_array[a] += prob[ns] * (rewards[ns] + gamma * s_value_function[ns])
        policy[s] = np.argmax(temp_array)
    return policy

In [150]:
def policy_iteration(dynamics, gamma=0.9, outer_iter=100, inner_iter=100):
    """
    optimizes a policy based on Policy Iteration
 
    Args:
        dynamics (FrozenLakeMDP): dynamics of the environment.
        gamma (float): discount factor.
        outer_iter (int): number of iterations for the Policy Iteration loop.
        inner_iter (int): number of iterations for the Policy Evaluation loop.
 
    Returns:
        numpy.array: the optimized policy.
    """

    # TODO: implement Policy Iteration algorithm
    policy = np.random.randint(0, 4, size=16)
    for i in range(outer_iter):
        state_values = policy_evaluation(dynamics, policy, inner_iter)
        policy = greedy_policy_improvement(dynamics, state_values)
    
    return policy

In [155]:
# TODO: test and analyze the algorithm

dynamics = FrozenLakeMDP(is_slippery=False)
policy = policy_iteration(dynamics)

In [156]:
# TODO: test the policy on the environment

env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode='rgb_array')
total_reward = record_video(env, policy, 'frozenlake_random.mp4', fps=5, random_action=False)
print(f"total reward: {total_reward}")
show_video('frozenlake_random.mp4', video_width=500)

action: 1, state: 4, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 1, state: 8, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 2, state: 9, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 1, state: 13, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 2, state: 14, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 2, state: 15, reward: 1.0, done: True, truncated: False, info: {'prob': 1.0}
total reward: 1.0


## Q-Learning

In [268]:
class QAgent:  # The Q-Learning RL agent

    def __init__(self, num_states, num_actions, epsilon, alpha, gamma=0.9, eps_end=0.01, eps_decay=3e-6):

        self.num_states = num_states    # number of possible states
        self.num_actions = num_actions  # number of possible actions
        self.gamma = gamma              # discount factor
        self.epsilon = epsilon          # initial exploration probability
        self.alpha = alpha              # step size
        self.eps_decay = eps_decay      # linear decay rate of epsilon
        self.eps_end = eps_end          # minimum value for epsilon
        self.q_table = np.zeros((num_states, num_actions), dtype=float)

    def choose_action(self, state):
        """
        chooses an action in an epsilon-greedy manner.
    
        Args:
            state (int): current state of the agent.
    
        Returns:
            int: the chosen action
        """
        
        # TODO: implement epsilon-greedy action selection

        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.num_actions)
        else:
            action = np.argmax(self.q_table[state])
            
        return action

    def learn(self, state, action, reward, next_state):
        """
        updates the q-table based on a single interaction with the environment.
    
        Args:
            state (int): state of the agent.
            action (int): action chosen by the agent.
            reward (int): reward obtained by the agent.
            next_state (int): next state of the agent.
        """

        # TODO: implement Q-table update
        old_Q = self.q_table[state][action]
        next_max_Q = np.max(self.q_table[next_state])
        
        new_Q = (1 - self.alpha) * old_Q + self.alpha * (reward + self.gamma * next_max_Q)
        self.q_table[state][action] = new_Q
        
        # epsilon decay
        self.epsilon = self.epsilon - self.eps_decay if self.epsilon > self.eps_end else self.eps_end

In [269]:
def train(env, agent, n_episodes=100000):
    """
        trains an agent through interactions with the environemnt using Q-learning.
    
        Args:
            env (gym.Env): the gym environment.
            agent (QAgent): the Q-learning agent.
            n_episodes (int): number of training episodes.
    """

    for i in range(n_episodes):

        # TODO: implement the training loop for Q-learning

        state = env.reset()[0]
        done = False
        
        while not done:
            action = agent.choose_action(state)
            next_state, reward, done, _, _ = env.step(action)
            agent.learn(state, action, reward, next_state)
            state = next_state

In [270]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False)
agent = QAgent(num_states=16, num_actions=4, epsilon=1.0, alpha=1e-3)

In [275]:
train(env, agent)

In [276]:
# TODO: obtain the policy by a simple argmax on agent's Q-table
policy = np.argmax(agent.q_table, axis=1)
print(policy)

[2 2 1 0 1 0 1 0 2 2 1 0 0 2 2 0]


In [278]:
# TODO: test the policy on the environment

env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode='rgb_array')
total_reward = record_video(env, policy, 'frozenlake_random.mp4', fps=5, random_action=False)
print(f"total reward: {total_reward}")
show_video('frozenlake_random.mp4', video_width=500)

action: 2, state: 1, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 2, state: 2, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 1, state: 6, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 1, state: 10, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 1, state: 14, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 2, state: 15, reward: 1.0, done: True, truncated: False, info: {'prob': 1.0}
total reward: 1.0
